# Segmentation

This notebook predicts the binary mask of squares/circles from RGB images.

In [1]:
import os
from torch import optim
from torch import nn

from cnn_framework.dummy_segmentation.data_set import DummyDataSet
from cnn_framework.dummy_segmentation.model_params import DummyModelParams
from cnn_framework.dummy_segmentation.model import UNet

from cnn_framework.utils.data_loader_generators.data_loader_generator import DataLoaderGenerator
from cnn_framework.utils.model_managers.model_manager import ModelManager
from cnn_framework.utils.data_managers.default_data_manager import DefaultDataManager
from cnn_framework.utils.metrics import PCC
from cnn_framework.utils.create_dummy_data_set import generate_data_set


In [2]:
params = DummyModelParams()
params.update()

# Create data set if needed
if not os.path.exists(params.data_dir):
    generate_data_set(params.data_dir)
    print(f"\nData set created in {params.data_dir}")

print(f"\nModel will be saved in {params.models_folder}")
print(f"Predictions will be saved in {params.output_dir}")
print(f"Tensorboard logs will be saved in {params.tensorboard_folder_path}")

Model time id: 20230904-104106-local
epochs 50 | batch 2 | lr 0.0001 | weight decay 0.0 | dropout 0.0

Model will be saved in C:\Users\thoma\models\local/dummy_segmentation/20230904-104106-local
Predictions will be saved in C:\Users\thoma\predictions\local/dummy_segmentation/20230904-104106-local
Tensorboard logs will be saved in C:\Users\thoma\tensorboard\local/20230904-104106-local_dummy_segmentation


In [3]:
loader_generator = DataLoaderGenerator(params, DummyDataSet, DefaultDataManager)
train_dl, val_dl, test_dl = loader_generator.generate_data_loader()

### Data source ###
train data is loaded from C:\Users\thoma\data\dummy - 80% elements
val data is loaded from C:\Users\thoma\data\dummy - 10% elements
test data is loaded from C:\Users\thoma\data\dummy - 10% elements
###################
train has 160 images.
val has 20 images.
test has 20 images.
###################


In [4]:
model = UNet(
    nb_classes=params.out_channels,
    nb_input_channels=params.nb_modalities * params.nb_stacks_per_modality,
)
manager = ModelManager(model, params, PCC)

optimizer = optim.Adam(
    model.parameters(),
    lr=float(params.learning_rate),
    betas=(params.beta1, params.beta2),
)  # define the optimization
loss_function = nn.L1Loss()

Current commit hash: 8fb09e49bd9849a488b1c0871e163fc6c713b2f7


In [5]:
manager.fit(train_dl, val_dl, optimizer, loss_function)

Training in progress: 100.0% | Local step 79 | Epoch 49
Best model saved at epoch 47.

Training successfully finished in 0:01:51.580172.


In [6]:
manager.predict(test_dl)

Model evaluation in progress: 100.0% | Batch #9
Average PCC: 1.0
